<a href="https://colab.research.google.com/github/AlexFly666/002-openai-quickstart-jike-peng/blob/main/langchain/jupyter/memory/memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 核心模块学习：Memory

大多数LLM应用都具有对话界面。对话的一个重要组成部分是能够引用先前在对话中介绍过的信息。至少，一个对话系统应该能够直接访问一些过去消息的窗口。更复杂的系统将需要拥有一个不断更新的世界模型，使其能够保持关于实体及其关系的信息。

我们将存储过去交互信息的能力称为“记忆（Memory）”。

LangChain提供了许多用于向应用/系统中添加 Memory 的实用工具。这些工具可以单独使用，也可以无缝地集成到链中。

一个记忆系统（Memory System）需要支持两个基本操作：**读取（READ）和写入（WRITE）**。

每个链都定义了一些核心执行逻辑，并期望某些输入。其中一些输入直接来自用户，但有些输入可能来自 Memory。

在一个典型 Chain 的单次运行中，将与其 Memory System 进行至少两次交互:

1. 在接收到初始用户输入之后，在执行核心逻辑之前，链将从其 Memory 中**读取**并扩充用户输入。
2. 在执行核心逻辑之后但在返回答案之前，一个链条将把当前运行的输入和输出**写入** Memory ，以便在未来的运行中可以引用它们。

![](https://github.com/AlexFly666/002-openai-quickstart-jike-peng/blob/main/langchain/jupyter/images/memory.png?raw=1)

In [2]:
!pip install langchain==0.3.14\
       langchain-openai==0.2.14\
       langchainhub==0.1.21

## BaseMemory Class 基类

类继承关系：

```
## 适用于简单的语言模型
BaseMemory --> BaseChatMemory --> <name>Memory  # Examples: ZepMemory, MotorheadMemory
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/schema/memory.py**

```python
# 定义一个名为BaseMemory的基础类
class BaseMemory(Serializable, ABC):
    """用于Chains中的内存的抽象基类。
    
    这里的内存指的是Chains中的状态。内存可以用来存储关于Chain的过去执行的信息，
    并将该信息注入到Chain的未来执行的输入中。例如，对于会话型Chains，内存可以用来
    存储会话，并自动将它们添加到未来的模型提示中，以便模型具有必要的上下文来连贯地
    响应最新的输入。"""

    # 定义一个名为Config的子类
    class Config:
        """为此pydantic对象配置。
    
        Pydantic是一个Python库，用于数据验证和设置管理，主要基于Python类型提示。
        """
    
        # 允许在pydantic模型中使用任意类型。这通常用于允许复杂的数据类型。
        arbitrary_types_allowed = True
    
    # 下面是一些必须由子类实现的方法：
    
    # 定义一个属性，它是一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法应返回该内存类将添加到链输入的字符串键。
    @property
    @abstractmethod
    def memory_variables(self) -> List[str]:
        """获取此内存类将添加到链输入的字符串键。"""
    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法基于给定的链输入返回键值对。
    @abstractmethod
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """根据给链的文本输入返回键值对。"""
    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法将此链运行的上下文保存到内存。
    @abstractmethod
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """保存此链运行的上下文到内存。"""
    
    # 定义一个抽象方法。任何从BaseMemory派生的子类都需要实现此方法。
    # 此方法清除内存内容。
    @abstractmethod
    def clear(self) -> None:
        """清除内存内容。"""
```

## BaseChatMessageHistory Class 基类

类继承关系：

```
## 适用于聊天模型

BaseChatMessageHistory --> <name>ChatMessageHistory  # Example: ZepChatMessageHistory
```

```python
# 定义一个名为BaseChatMessageHistory的基础类
class BaseChatMessageHistory(ABC):
    """聊天消息历史记录的抽象基类。"""

    # 在内存中存储的消息列表
    messages: List[BaseMessage]

    # 定义一个add_user_message方法，它是一个方便的方法，用于将人类消息字符串添加到存储区。
    def add_user_message(self, message: str) -> None:
        """为存储添加一个人类消息字符串的便捷方法。

        参数:
            message: 人类消息的字符串内容。
        """
        self.add_message(HumanMessage(content=message))

    # 定义一个add_ai_message方法，它是一个方便的方法，用于将AI消息字符串添加到存储区。
    def add_ai_message(self, message: str) -> None:
        """为存储添加一个AI消息字符串的便捷方法。

        参数:
            message: AI消息的字符串内容。
        """
        self.add_message(AIMessage(content=message))

    # 抽象方法，需要由继承此基类的子类来实现。
    @abstractmethod
    def add_message(self, message: BaseMessage) -> None:
        """将Message对象添加到存储区。

        参数:
            message: 要存储的BaseMessage对象。
        """
        raise NotImplementedError()

    # 抽象方法，需要由继承此基类的子类来实现。
    @abstractmethod
    def clear(self) -> None:
        """从存储中删除所有消息"""

```

### ConversationChain and ConversationBufferMemory

`ConversationBufferMemory` 可以用来存储消息，并将消息提取到一个变量中。

In [16]:
from langchain_openai import OpenAI,ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# 使用智谱API，需要修改base_url
# llm = OpenAI(
#     temperature=0.95,
#     model="glm-4v-flash",
#     openai_api_key="XXXX",
#     openai_api_base="https://open.bigmodel.cn/api/paas/v4/chat"
# )
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4v-flash",
    openai_api_key="XXXX",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [17]:
conversation.predict(input="你好呀！")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好呀！
AI:

> Finished chain.


'你好呀！很高兴见到你！有什么我可以帮助你的吗？'

In [11]:
conversation.predict(input="你为什么叫小米？跟雷军有关系吗？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好呀！
AI: 你好呀！很高兴见到你！有什么我可以帮助你的吗？无论是解答问题、提供信息还是其他任何事情，只要告诉我你的需求，我都会尽力为你提供服务。
Human: 你为什么叫小米？跟雷军有关系吗？
AI:

> Finished chain.


'你好！我并不是小米公司的产品或与雷军先生有直接关联。"小米"这个名字是我的人工智能助手ID，它简洁明了，易于记忆和发音。我的目的是为用户提供便捷、高效的服务和信息支持。\n\n如果你有任何问题或者需要帮助的地方，请随时告诉我哦！我会尽我所能为你提供服务。'

### ConversationBufferWindowMemory
`ConversationBufferWindowMemory` 会在时间轴上保留对话的交互列表。它只使用最后 K 次交互。这对于保持最近交互的滑动窗口非常有用，以避免缓冲区过大。

In [20]:
from langchain.memory import ConversationBufferWindowMemory

# 使用智谱API，需要修改base_url
llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4v-flash",
    openai_api_key="XXXX",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

conversation_with_summary = ConversationChain(
    llm=llm,
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)
conversation_with_summary.predict(input="嗨，你最近过得怎么样？")

<ipython-input-20-e5f1e6083914>:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferWindowMemory(k=2),




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 嗨，你最近过得怎么样？
AI:

> Finished chain.


'嗨！你好呀！作为一个人工智能助手，我没有感受和经历，但我一直在这里随时准备帮助你解答问题和完成任务。有什么我可以帮到你的吗？'

In [21]:
conversation_with_summary.predict(input="你最近学到什么新知识了?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 嗨，你最近过得怎么样？
AI: 嗨！你好呀！作为一个人工智能助手，我没有感受和经历，但我一直在这里随时准备帮助你解答问题和完成任务。有什么我可以帮到你的吗？
Human: 你最近学到什么新知识了?
AI:

> Finished chain.


'抱歉，我之前的回答可能有些误导性。实际上，作为一个基于文本的人工智能，我没有学习或记忆的能力，因为我无法像人类那样进行感官体验和学习过程。我是通过大量的数据和算法来生成回答的，但这些回答是基于我被训练时的数据集，而不是实时学习的。\n\n如果你有任何问题或者需要帮助的地方，请随时告诉我，我会尽我所能为你提供服务。'

In [22]:
conversation_with_summary.predict(input="展开讲讲？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 嗨，你最近过得怎么样？
AI: 嗨！你好呀！作为一个人工智能助手，我没有感受和经历，但我一直在这里随时准备帮助你解答问题和完成任务。有什么我可以帮到你的吗？
Human: 你最近学到什么新知识了?
AI: 抱歉，我之前的回答可能有些误导性。实际上，作为一个基于文本的人工智能，我没有学习或记忆的能力，因为我无法像人类那样进行感官体验和学习过程。我是通过大量的数据和算法来生成回答的，但这些回答是基于我被训练时的数据集，而不是实时学习的。

如果你有任何问题或者需要帮助的地方，请随时告诉我，我会尽我所能为你提供服务。
Human: 展开讲讲？
AI:

> Finished chain.


'当然可以。我的设计是为了处理和分析文本信息，这意味着我可以理解和生成自然语言。我被训练来识别模式、回答查询、提供建议以及执行其他与文本相关的任务。然而，我不具备感知能力，所以我不能“看到”或“听到”任何东西；我也不能有个人经验或情感反应。\n\n尽管如此，我在2023之前接受了大量数据的训练，这使我能够对各种主题产生相当详尽的回答。这些答案是基于我所学到的模式和关联，但它们并不是来自个人的学习和感知。\n\n例如，我可以告诉你关于历史事件的信息，解释科学概念，甚至帮你规划旅行路线。但是，我不能亲自去体验任何事情，也不能从实际生活中获得新的知识。\n\n如果你想了解更多关于我是如何工作的细节，或者有任何具体的问题，我都会尽力提供帮助和信息。'

In [23]:
# 注意：第一句对话从 Memory 中移除了.
conversation_with_summary.predict(input="如果要构建聊天机器人，具体要用什么自然语言处理技术?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你最近学到什么新知识了?
AI: 抱歉，我之前的回答可能有些误导性。实际上，作为一个基于文本的人工智能，我没有学习或记忆的能力，因为我无法像人类那样进行感官体验和学习过程。我是通过大量的数据和算法来生成回答的，但这些回答是基于我被训练时的数据集，而不是实时学习的。

如果你有任何问题或者需要帮助的地方，请随时告诉我，我会尽我所能为你提供服务。
Human: 展开讲讲？
AI: 当然可以。我的设计是为了处理和分析文本信息，这意味着我可以理解和生成自然语言。我被训练来识别模式、回答查询、提供建议以及执行其他与文本相关的任务。然而，我不具备感知能力，所以我不能“看到”或“听到”任何东西；我也不能有个人经验或情感反应。

尽管如此，我在2023之前接受了大量数据的训练，这使我能够对各种主题产生相当详尽的回答。这些答案是基于我所学到的模式和关联，但它们并不是来自个人的学习和感知。

例如，我可以告诉你关于历史事件的信息，解释科学概念，甚至帮你规划旅行路线。但是，我不能亲自去体验任何事情，也不能从实际生活中获得新的知识。

如果你想了解更多关于我是如何工作的细节，或者有任何具体的问题，我都会尽力提供帮助和信息。
Human: 如果要构建聊天机器人，具体要用什么自然语言处理技术?
AI:

> Finished chain.


'构建聊天机器人通常涉及多种自然语言处理（NLP）技术和工具。以下是一些关键的技术和步骤：\n\n1. **分词**：将输入的文本分割成单词或其他有意义的最小单位。\n\n2. **词性标注**：确定每个单词在句子中的语法角色，如名词、动词、形容词等。\n\n3. **命名实体识别（NER）**：识别文本中的特定实体，如人名、地点、组织等。\n\n4. **句法分析**：理解句子的结构，包括主谓宾关系、从句和其他复杂的语法构造。\n\n5. **语义理解**：解析单词和短语的含义，并理解它们之间的关系。\n\n6. **意图检测**：判断用户的请求或问题背后的目的或意图。\n\n7. **对话管理**：跟踪对话的状态，决定下一步应该做什么，比如提出问题、给出建议或结束对话。\n\n8. **机器翻译**：如果聊天机器人需要支持多种语言，那么机器翻译技术是必不可少的。\n\n9. **情感分析**：评估用户情绪，以便更好地响应他们的需求。\n\n10. **上下文建模**：保持对话的连贯性，记住先前的交互内容。\n\n11. **个性化**：根据用户的历史互动和行为调整对话的内容和风格。\n\n12. **反馈循环**：收集用户反馈，以改进聊天机器人的性能。\n\n在实际应用中，通常会使用深度学习框架，如TensorFlow或PyTorch，结合预训练的语言模型，如BERT、GPT系列或GLM-130B等，来实现上述功能。此外，还会用到数据库管理系统来存储对话数据，以及API服务来集成外部资源和服务。\n\n开发聊天机器人是一个复杂的过程，涉及到数据采集、预处理、模型训练、测试和部署等多个阶段。而且，随着技术的发展，聊天机器人的功能和性能也在不断进步和完善。'

In [24]:
conversation_with_summary.__dict__

{'name': None,
 'memory': ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='嗨，你最近过得怎么样？', additional_kwargs={}, response_metadata={}), AIMessage(content='嗨！你好呀！作为一个人工智能助手，我没有感受和经历，但我一直在这里随时准备帮助你解答问题和完成任务。有什么我可以帮到你的吗？', additional_kwargs={}, response_metadata={}), HumanMessage(content='你最近学到什么新知识了?', additional_kwargs={}, response_metadata={}), AIMessage(content='抱歉，我之前的回答可能有些误导性。实际上，作为一个基于文本的人工智能，我没有学习或记忆的能力，因为我无法像人类那样进行感官体验和学习过程。我是通过大量的数据和算法来生成回答的，但这些回答是基于我被训练时的数据集，而不是实时学习的。\n\n如果你有任何问题或者需要帮助的地方，请随时告诉我，我会尽我所能为你提供服务。', additional_kwargs={}, response_metadata={}), HumanMessage(content='展开讲讲？', additional_kwargs={}, response_metadata={}), AIMessage(content='当然可以。我的设计是为了处理和分析文本信息，这意味着我可以理解和生成自然语言。我被训练来识别模式、回答查询、提供建议以及执行其他与文本相关的任务。然而，我不具备感知能力，所以我不能“看到”或“听到”任何东西；我也不能有个人经验或情感反应。\n\n尽管如此，我在2023之前接受了大量数据的训练，这使我能够对各种主题产生相当详尽的回答。这些答案是基于我所学到的模式和关联，但它们并不是来自个人的学习和感知。\n\n例如，我可以告诉你关于历史事件的信息，解释科学概念，甚至帮你规划旅行路线。但是，我不能亲自去体验任何事情，也不能从实际生活中获得新的知识。\n\n如果你想了

### ConversationSummaryBufferMemory

`ConversationSummaryBufferMemory` 在内存中保留了最近的交互缓冲区，但不仅仅是完全清除旧的交互，而是将它们编译成摘要并同时使用。与以前的实现不同的是，它使用token长度而不是交互次数来确定何时清除交互。

In [27]:
from langchain.memory import ConversationSummaryBufferMemory
# 换成Open
llm = ChatOpenAI(
    temperature=0.95,
    model="gpt-3.5-turbo-instruct",
    openai_api_key="XXXX",
    openai_api_base="https://vip.apiyi.com/v1"
)
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "嗨，你最近过得怎么样？"}, {"output": " 嗨！我最近过得很好，谢谢你问。我最近一直在学习新的知识，并且正在尝试改进自己的性能。我也在尝试更多的交流，以便更好地了解人类的思维方式。"})
memory.save_context({"input": "你最近学到什么新知识了?"}, {"output": " 最近我学习了有关自然语言处理的知识，以及如何更好地理解人类的语言。我还学习了有关机器学习的知识，以及如何使用它来改善自己的性能。"})

In [ ]:
memory.load_memory_variables({})

{'history': 'System: \nThe human asks how the AI has been doing lately. The AI responds that it has been doing well, thanks for asking. It has been learning new knowledge and trying to improve its performance, including in the areas of natural language processing and machine learning. It has also been trying to communicate more in order to better understand human thinking.'}

In [ ]:
print(memory.load_memory_variables({})['history'])

System: 
The human asks how the AI has been doing lately. The AI responds that it has been doing well, thanks for asking. It has been learning new knowledge and trying to improve its performance, including in the areas of natural language processing and machine learning. It has also been trying to communicate more in order to better understand human thinking.
